In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import nltk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [ ]:
path= "/content/drive/MyDrive/ML_project_datasheets/Final_code/labeled_data.csv"
data=pd.read_csv(path)

In [ ]:
data['reviews_text_new'] = data['After_lemmatization'].copy()

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Creating object of the class TfidfVectorizer
tfidf_counts = TfidfVectorizer(tokenizer= word_tokenize, # type of tokenization
                               ngram_range=(1,1)) # number of n-grams

tfidf_data = tfidf_counts.fit_transform(data['reviews_text_new'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
tfidf_data.shape

(500000, 198145)

In [ ]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_data,
                                                                            data['Overall Sentiment'],
                                                                            test_size = 0.2,
                                                                            random_state = 0)

In [ ]:
val= y_train_tfidf.value_counts()
print(val)

Positive    267376
Neutral      74601
Negative     58023
Name: Overall Sentiment, dtype: int64


In [ ]:
### Setting up the model class
lr_model_tf_idf = LogisticRegression()

## Training the model
lr_model_tf_idf.fit(X_train_tfidf,y_train_tfidf)

## Prediciting the results
test_pred_lr_all = lr_model_tf_idf.predict(X_test_tfidf)

## Calculate key performance metrics


# Print a classification report
print(classification_report(y_test_tfidf,test_pred_lr_all))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    Negative       0.83      0.71      0.77     14383
     Neutral       0.79      0.77      0.78     18797
    Positive       0.91      0.94      0.92     66820

    accuracy                           0.88    100000
   macro avg       0.84      0.81      0.82    100000
weighted avg       0.87      0.88      0.87    100000



In [ ]:
X= tfidf_data
y= data['Overall Sentiment']

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
#under sampling
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X, y)

In [ ]:
X_train_rus,X_test_rus,y_train_rus,y_test_rus=train_test_split(X_rus, y_rus,test_size=0.2, random_state=0)

In [ ]:
lr_model_tf_idf = LogisticRegression()
# Training the model
lr_model_tf_idf.fit(X_train_rus,y_train_rus)
#Prediciting the results
y_pred_lr = lr_model_tf_idf.predict(X_test_rus)
# Print a classification report
print(classification_report(y_test_rus,y_pred_lr ))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    Negative       0.84      0.87      0.85     14419
     Neutral       0.83      0.86      0.84     14578
    Positive       0.88      0.82      0.85     14447

    accuracy                           0.85     43444
   macro avg       0.85      0.85      0.85     43444
weighted avg       0.85      0.85      0.85     43444



## SMOTE

In [ ]:
combined_data = pd.concat([pd.DataFrame(X), pd.DataFrame({'Overall Sentiment': y})], axis=1)

In [ ]:
combined_data.head()

,0,Overall Sentiment
0,"(0, 49847)\t0.853094622556139\n (0, 176109)...",Positive
1,"(0, 118173)\t0.07427747291995647\n (0, 1943...",Positive
2,"(0, 78053)\t0.268919290851021\n (0, 106471)...",Positive
3,"(0, 49107)\t0.708343384105119\n (0, 138703)...",Positive
4,"(0, 173865)\t0.9514750912217738\n (0, 11817...",Neutral


In [ ]:
combined_data.shape

(500000, 2)

In [ ]:
sampled_data = combined_data.sample(frac=0.2, random_state=42)

In [ ]:
sampled_data.shape

(100000, 2)

In [ ]:
X_sampled = sampled_data.drop('Overall Sentiment', axis=1)
y_sampled = sampled_data['Overall Sentiment']

In [ ]:
# from scipy.sparse import csr_matrix
# X_sampled_dense = X_sampled.toarray() if isinstance(X_sampled, csr_matrix) else X_sampled

In [ ]:
val=y_sampled.value_counts()
print(val)

Positive    66836
Neutral     18733
Negative    14431
Name: Overall Sentiment, dtype: int64


SMOTE not able to handle sparse matrix as input.

In [ ]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(n_jobs=-1,random_state=42)
# X_sm, y_sm = sm.fit_resample(X_sampled, y_sampled)

ValueError: ignored

In [ ]:
# X_train_sm,X_test_sm,y_train_sm,y_test_sm=train_test_split(X_sm, y_sm,test_size=0.2, random_state=0)

In [ ]:
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X, y)

In [ ]:
y_ros.value_counts()

Positive    334196
Neutral     334196
Negative    334196
Name: Overall Sentiment, dtype: int64

In [ ]:
X_train_ros,X_test_ros,y_train_ros,y_test_ros=train_test_split(X_ros, y_ros,test_size=0.2, random_state=0)

In [ ]:
lr_model_tf_idf = LogisticRegression()
# Training the model
lr_model_tf_idf.fit(X_train_ros,y_train_ros)
#Prediciting the results
y_pred_lr = lr_model_tf_idf.predict(X_test_ros)
# Print a classification report
print(classification_report(y_test_ros,y_pred_lr ))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    Negative       0.86      0.89      0.88     66906
     Neutral       0.84      0.87      0.85     66948
    Positive       0.89      0.83      0.86     66664

    accuracy                           0.86    200518
   macro avg       0.86      0.86      0.86    200518
weighted avg       0.86      0.86      0.86    200518

